In [1]:
import pandas as pd
orders = pd.read_parquet("orders.parquet")  
order_products_denormalized = pd.read_csv("order_products_denormalized.csv")  
tips_public = pd.read_csv("tips_public.csv").drop(columns=["Unnamed: 0"])  

In [2]:
orders.head(1) 

,order_id,user_id,order_date
0,1374495,3,2024-03-31 14:05:18


In [3]:
tips_public.head(1)

,order_id,tip
0,1374495,True


In [4]:
order_products_denormalized_sample = order_products_denormalized[order_products_denormalized["order_id"] < 10000]
order_products_denormalized.tail(1)

,order_id,product_id,add_to_cart_order,product_name,aisle_id,department_id,department,aisle
14857352,3421083,5020,10,Organic Sweet & Salty Peanut Pretzel Granola ...,3,19,snacks,energy granola bars


### Feature engineering **USER**: new table per feature with columns: user_id, feature_name


In [5]:
def add_feature_alcohol_count():
    # 1. Filter products in the "alcohol" department
    alcohol_df = order_products_denormalized[order_products_denormalized["department"] == "alcohol"]

    # 2. Join with the 'orders' table to get user_id for each order
    alcohol_with_users = alcohol_df.merge(orders[["order_id", "user_id"]], on="order_id")

    # 3. Count how many times each user bought alcohol
    alcohol_counts = alcohol_with_users["user_id"].value_counts().reset_index()
    alcohol_counts.columns = ["user_id", "alcohol_purchases"]

    # Optional: sort users by alcohol purchases (descending)
    alcohol_counts = alcohol_counts.sort_values(by="alcohol_purchases", ascending=False)

    # 4. Merge with all user_ids to include users with 0 alcohol purchases
    result = (
        alcohol_counts
        .merge(orders[["user_id"]], how="right")
        .drop_duplicates()
        .fillna(0)
        .set_index("user_id")
    )

    return result
   
#add_feature_alcohol_count().head(1)

In [ ]:
def total_products_per_user():
    # Join order_products_denormalized mit orders, um user_id zu bekommen
    merged = order_products_denormalized.merge(orders[["order_id", "user_id"]], on="order_id")
    
    # Für jeden user_id die Gesamtanzahl der Produkte zählen (inkl. Duplikate)
    total_products = merged.groupby("user_id")["product_id"].count().reset_index()
    total_products.columns = ["user_id", "total_products_bought"]
    
    return total_products
#total_products_per_user().head(1)

,user_id,total_products_bought
0,3,88


In [ ]:
def total_unique_products_per_user():
    """
    Calculate how many unique products each user has purchased in total.
    """
    merged = order_products_denormalized.merge(orders[["order_id", "user_id"]], on="order_id")
    unique_products = (
        merged.groupby("user_id")["product_id"]
        .nunique()
        .reset_index()
        .rename(columns={"product_id": "unique_products_bought"})
    )
    return unique_products
#total_unique_products_per_user().head(1)

,user_id,unique_products_bought
0,3,33


In [ ]:
def unique_to_total_product_ratio_per_user():
    """
    Calculate the ratio of unique products bought to total products bought per user.
    This shows how diverse the user's purchases are relative to their total quantity.
    """
    total = total_products_per_user()
    unique = total_unique_products_per_user()
    
    merged = total.merge(unique, on="user_id")
    merged["unique_to_total_product_ratio"] = merged["unique_products_bought"] / merged["total_products_bought"]
    
    return merged[["user_id", "unique_to_total_product_ratio"]]
#unique_to_total_product_ratio_per_user().head(1)

,user_id,unique_to_total_product_ratio
0,3,0.375


### Feature engineering **ORDER**: new table per feature with columns: order_id, feature_name


In [9]:
def add_feature_order_contains_alcohol():
    # Step 1: Identify all order_ids that include alcohol products
    alcohol_orders = order_products_denormalized[
        order_products_denormalized["department"] == "alcohol"
    ][["order_id"]].drop_duplicates()
    
    # Step 2: Mark these orders with a True flag
    alcohol_orders["contains_alcohol"] = True

    # Step 3: Get all unique orders and join with the alcohol flag
    all_orders = orders[["order_id"]].drop_duplicates()
    result = all_orders.merge(alcohol_orders, on="order_id", how="left")

    # Step 4: Fill missing values (orders without alcohol) with False (0)
    result["contains_alcohol"] = result["contains_alcohol"].fillna(False).astype(int)

    return result.set_index("order_id")

#add_feature_order_contains_alcohol().head(1)

In [10]:
def add_feature_order_item_count():
    # Count how many products each order contains
    item_counts = (
        order_products_denormalized
        .groupby("order_id")["product_id"]
        .count()
        .reset_index()
        .rename(columns={"product_id": "item_count"})
    )
    return item_counts

#add_feature_order_item_count().head(1)

In [11]:
def add_feature_order_unique_departments_count():
    # Count distinct departments per order_id
    dept_counts = (
        order_products_denormalized
        .groupby("order_id")["department"]
        .nunique()
        .reset_index()
        .rename(columns={"department": "unique_departments_count"})
    )
    return dept_counts#.set_index("order_id")

#add_feature_order_unique_departments_count().head(1)

In [ ]:
def add_feature_order_unique_aisles_count():
    # Count distinct aisles per order_id
    aisle_counts = (
        order_products_denormalized
        .groupby("order_id")["aisle"]
        .nunique()
        .reset_index()
        .rename(columns={"aisle": "unique_aisles_count"})
    )
    return aisle_counts
#add_feature_order_unique_aisles_count().head(1)

,order_id,unique_aisles_count
0,1,6


In [ ]:
def add_feature_order_unique_departments_ratio():
    """
    Calculate the ratio of unique departments to total items per order. \n
    This feature reflects how diverse an order is relative to its size.
    """
    total_items = add_feature_order_item_count()
    unique_depts = add_feature_order_unique_departments_count()

    # Merge on order_id
    merged = total_items.merge(unique_depts, on="order_id")

    # Calculate ratio
    merged["unique_departments_ratio"] = merged["unique_departments_count"] / merged["item_count"]

    return merged[["order_id", "unique_departments_ratio"]]
#add_feature_order_unique_departments_ratio().head(1)

,order_id,unique_departments_ratio
0,1,0.375


In [ ]:
def add_feature_order_unique_aisles_ratio():
    """
    Calculate the ratio of unique aisles to total items per order. \n
    This feature indicates the variety of product aisles in the order normalized by order size.
    """
    total_items = add_feature_order_item_count()
    unique_aisles = add_feature_order_unique_aisles_count()

    merged = total_items.merge(unique_aisles, on="order_id")

    merged["unique_aisles_ratio"] = merged["unique_aisles_count"] / merged["item_count"]

    return merged[["order_id", "unique_aisles_ratio"]]
#add_feature_order_unique_aisles_ratio().head(1)

,order_id,unique_aisles_ratio
0,1,0.75


### Feature engineering **ORDER** - **USER**: new table per feature with columns: order_id, feature_name

In [33]:
def add_cumulative_product_purchases_per_user():
    # 1. Merge orders und order_products_denormalized, damit wir user_id, product_id und order_date haben
    merged = order_products_denormalized.merge(
        orders[["order_id", "user_id", "order_date"]], on="order_id"
    )

    # 2. Sortiere nach user_id, product_id und order_date, damit der Zeitverlauf stimmt
    merged = merged.sort_values(by=["user_id", "product_id", "order_date"])

    # 3. Erstelle eine kumulative Zählung (cumcount) pro user_id und product_id
    #    cumcount zählt ab 0, also entspricht es, wie oft der User das Produkt vor der aktuellen Bestellung gekauft hat
    merged["times_product_bought_before"] = merged.groupby(
        ["user_id", "product_id"]
    ).cumcount()

    return merged
#add_cumulative_product_purchases_per_user().head(1)

In [23]:
def latest_tip_rate_per_user_product():
    """
    Return the latest purchase per user-product combination with the tip rate
    observed on *prior* purchases of that product by the user.
    If it's the first time, the value will be NaN.
    """
    merged = order_products_denormalized.merge(
        orders[["order_id", "user_id", "order_date"]], on="order_id"
    ).merge(
        tips_public[["order_id", "tip"]], on="order_id", how="left"
    )

    merged = merged.sort_values(by=["user_id", "product_id", "order_date"])

    merged["tip"] = merged["tip"].fillna(False).astype(float)

    # Count how often user bought the product before
    merged["times_product_bought_before"] = merged.groupby(
        ["user_id", "product_id"]
    ).cumcount()

    # How many of those times included a tip
    merged["tip_cumsum"] = merged.groupby(
        ["user_id", "product_id"]
    )["tip"].cumsum() - merged["tip"]

    merged["tip_rate_before"] = merged["tip_cumsum"] / merged["times_product_bought_before"]

    # Only keep the most recent row for each (user_id, product_id)
    latest = merged.sort_values(by="order_date").groupby(
        ["user_id", "product_id"], as_index=False
    ).last()

    return latest[["user_id", "product_id", "tip_rate_before"]]
#latest_tip_rate_per_user_product().head(6)

In [ ]:
def add_avg_tip_rate_before_per_order():
    """
    For each order, calculate the average historical tip rate of all products
    in that order (based on user's previous purchases).
    This is useful as a predictive feature for whether a tip will be given.
    """
    # Merge all relevant info
    merged = order_products_denormalized.merge(
        orders[["order_id", "user_id", "order_date"]], on="order_id"
    ).merge(
        tips_public[["order_id", "tip"]], on="order_id", how="left"
    )

    merged = merged.sort_values(by=["user_id", "product_id", "order_date"])

    merged["tip"] = merged["tip"].fillna(0.0).astype(float)

    # Calculate how often user bought product before
    merged["times_product_bought_before"] = merged.groupby(
        ["user_id", "product_id"]
    ).cumcount()

    # How many of those had tips
    merged["tip_cumsum"] = merged.groupby(
        ["user_id", "product_id"]
    )["tip"].cumsum() - merged["tip"]

    merged["tip_rate_before"] = merged["tip_cumsum"] / merged["times_product_bought_before"]
    merged["tip_rate_before"] = merged["tip_rate_before"].replace([float("inf"), -float("inf")], pd.NA)

    # Now compute average per order (ignoring NaNs for first-time products)
    avg_tip_rate_per_order = merged.groupby("order_id")["tip_rate_before"].mean().reset_index()
    avg_tip_rate_per_order.columns = ["order_id", "avg_tip_rate_before"]

    return avg_tip_rate_per_order
avg_tip_rate_before_per_order = add_avg_tip_rate_before_per_order()
avg_tip_rate_before_per_order.head(1)

In [47]:
orders[orders["user_id"] == 5].merge(order_products_denormalized).merge(tips_public).merge(avg_tip_rate_before_per_order)

,order_id,user_id,order_date,product_id,add_to_cart_order,product_name,aisle_id,department_id,department,aisle,tip,avg_tip_rate_before
0,2717275,5,2024-04-02 12:01:12,15349,1,Organic Raw Agave Nectar,29,13,pantry,honeys syrups nectars,False,NaN
1,2717275,5,2024-04-02 12:01:12,21413,2,Organic Soba,66,6,international,asian foods,False,NaN
2,2717275,5,2024-04-02 12:01:12,48775,3,Organic Red Cabbage,83,4,produce,fresh vegetables,False,NaN
3,2717275,5,2024-04-02 12:01:12,28289,4,Organic Shredded Carrots,123,4,produce,packaged vegetables fruits,False,NaN
4,2717275,5,2024-04-02 12:01:12,8518,5,Organic Red Onion,83,4,produce,fresh vegetables,False,NaN
5,2717275,5,2024-04-02 12:01:12,11777,6,Red Raspberries,123,4,produce,packaged vegetables fruits,False,NaN
6,2717275,5,2024-04-02 12:01:12,31717,7,Organic Cilantro,16,4,produce,fresh herbs,False,NaN
7,2717275,5,2024-04-02 12:01:12,26604,8,Organic Blackberries,24,4,produce,fresh fruits,False,NaN
8,2717275,5,2024-04-02 12:01:12,43693,9,Whole Vitamin D Milk,84,16,dairy eggs,milk,False,NaN
9,2717275,5,2024-04-02 12:01:12,22475,10,Jicama,83,4,produce,fresh vegetables,False,NaN


In [50]:
def add_avg_tip_rate_before_per_product_row():
    """
    Add a column to each order-product row that contains the average historical tip rate
    the user had given for this specific product in the past. If the product is new, it's NaN.
    Useful as a contextual feature for tipping behavior on a product level.
    """
    # 1. Merge necessary data
    merged = order_products_denormalized.merge(
        orders[["order_id", "user_id", "order_date"]], on="order_id"
    ).merge(
        tips_public[["order_id", "tip"]], on="order_id", how="left"
    )

    # 2. Sort to ensure chronological order for cumsum logic
    merged = merged.sort_values(by=["user_id", "product_id", "order_date"])

    # 3. Ensure tip is numeric
    merged["tip"] = merged["tip"].fillna(0.0).astype(float)

    # 4. Count how many times the user has bought this product before this order
    merged["times_product_bought_before"] = merged.groupby(
        ["user_id", "product_id"]
    ).cumcount()

    # 5. Cumulative sum of tips before this point
    merged["tip_cumsum"] = merged.groupby(["user_id", "product_id"])["tip"].cumsum() - merged["tip"]

    # 6. Compute historical tip rate before for this product
    merged["avg_tip_rate_before_product"] = (
        merged["tip_cumsum"] / merged["times_product_bought_before"]
    )

    # 7. Handle division by 0 (new products)
    merged["avg_tip_rate_before_product"] = merged["avg_tip_rate_before_product"].replace(
        [float("inf"), -float("inf")], pd.NA
    )

    return merged
add_avg_tip_rate_before_per_product_row().sort_values("order_date").head(60)

,order_id,product_id,add_to_cart_order,product_name,aisle_id,department_id,department,aisle,user_id,order_date,tip,times_product_bought_before,tip_cumsum,avg_tip_rate_before_product
6792069,1510472,10121,9,Wild Sockeye Salmon,15,12,meat seafood,packaged seafood,87063,2024-01-17 00:17:26,1.0,0,0.0,NaN
6792063,1510472,12745,3,"Yogurt, Lowfat, Strawberry",120,16,dairy eggs,yogurt,87063,2024-01-17 00:17:26,1.0,0,0.0,NaN
6792066,1510472,14161,6,Kiwi,24,4,produce,fresh fruits,87063,2024-01-17 00:17:26,1.0,0,0.0,NaN
6792075,1510472,15741,15,Strawberry C Monster Smoothie,31,7,beverages,refrigerated,87063,2024-01-17 00:17:26,1.0,0,0.0,NaN
6792062,1510472,19348,2,Fat Free Milk,84,16,dairy eggs,milk,87063,2024-01-17 00:17:26,1.0,0,0.0,NaN
6792065,1510472,24852,5,Banana,24,4,produce,fresh fruits,87063,2024-01-17 00:17:26,1.0,0,0.0,NaN
6792074,1510472,28945,14,Honey Lemon with Echinacea Cough Suppressant T...,11,11,personal care,cold flu allergy,87063,2024-01-17 00:17:26,1.0,0,0.0,NaN
6792071,1510472,29987,11,Shredded Mozzarella,21,16,dairy eggs,packaged cheese,87063,2024-01-17 00:17:26,1.0,0,0.0,NaN
6792073,1510472,22256,13,Organic Throat Comfort Tea,94,7,beverages,tea,87063,2024-01-17 00:17:26,1.0,0,0.0,NaN
6792067,1510472,33787,7,Total 2% Lowfat Greek Strained Yogurt with Peach,120,16,dairy eggs,yogurt,87063,2024-01-17 00:17:26,1.0,0,0.0,NaN
